In [0]:
!pip install openai==0.28

In [0]:
dbutils.library.restartPython()

In [0]:
import openai
openai.api_type = "azure"
openai.api_base = "https://oai-dk.openai.azure.com/"
openai.api_version = "2023-12-01-preview"
openai.api_key = dbutils.secrets.get('akvdesafiokinea','azure-oai-dk')

In [0]:
#Função para Gerar Restrição
def gerar_restricao(dic):
    fundo = dic['IdFundo']
    valor_base = dic['SaldoBaseCalculo']
    exposicao = dic['ValorExposicao']
    valor_objeto = dic['SaldoObjeto']
    l_min = dic['LimiteMin']
    l_max = dic['LimiteMax']
    descricao = dic['DescricaoDetalhada']
    #Pegar ativo pela memória de cálculo -> Usar ativo e fundo para localizar qual posição na régua original vai ser afetada pela restrição
    message_text = [
                {"role":"system",
                "content":
                    """
                    Você receberá uma descrição de uma regra que foi descumprida, também receberá os limites mínimos e máximos referentes a está mesma regra. Assim como o valor base que pode ser o Patrimônio Líquido do fundo e o valor objeto que é o valor que está tentando ser alocado naquele fundo, o valor exposição é calculado em torno do valor objeto e do valor base e ele representa um percentual que foi restrito pela regra. Seu objetivo é gerar uma função de restrição no estilo scipy no formato:
                    def restricao(x):
                        return x
                    Lembre-se que x é um valor monetário igual valor objeto e não um percentual, sua restrição deve considerar o calculo. Como também considerar os limites mínimos e máximos.
                    Também retorne um texto escrito "eq" ou "ineq" a depender do tipo de restrição que estiver usando.  
                """
                },
                {"role":"user","content":f"Descrição: {descricao}; Limite Mínimo: {l_min}; Limite Máximo: {l_max}; Valor Base: {valor_base}; Valor Objeto: {valor_objeto}, Valor Exposição: {exposicao}"}
               ]
    
    completion = openai.ChatCompletion.create(
        engine="gpt35turbo16k",
        messages = message_text,
        temperature=0,
        max_tokens=1200,
        top_p=0.8,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None)

    return completion.to_dict()['choices'][0]['message']['content']

In [0]:
df_teste = spark.sql("""
                        select IdFundo,DataHoraVersao,h.IdRegra,SaldoBaseCalculo,ValorExposicao,SaldoObjeto,LimiteMin,LimiteMax,DescricaoDetalhada,h.Status,ComplementoId 
                        from desafio_kinea.boletagem_cp.nxenq_resultadoenquadramentohist as h LEFT JOIN desafio_kinea.boletagem_cp.nxenq_regras as r 
                        ON  SUBSTRING(h.IdRegra,2,LEN(h.IdRegra)) = r.IdRegra
                        where h.DataHoraVersao = '2025-02-06T11:48:25.000+00:00' and status = 1
                     """)
exemplo = df_teste.toPandas().iloc[5]
resposta = gerar_restricao(exemplo)
print(resposta)

In [0]:
ex_dic= {'ativo': 'UNEG11', 'pior_rating_permitido': 2, 'rating do ativo': 6, 'posicao_ativo': 118787475.0} #A partir do Parser
ex_regra = "Vedação - Ativos de Crédito Privado c/ Rating Diferente de A" #A partir de nxenq_regras.Descricao

resposta = gerar_restricao(ex_regra,ex_dic)
print(resposta)


In [0]:
%sql
select * from desafio_kinea.boletagem_cp.nxenq_resultadoenquadramentohist where status = 1 and DataHoraVersao = '2025-02-07T11:22:56.000+00:00'